In [ ]:

import numpy as np 
import pandas as pd 


import re
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.metrics.pairwise import linear_kernel

import os


In [2]:
data = pd.read_csv("/kaggle/input/processed-all-datas/Processed_all_datas.csv")
data.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_date
0,US,51632887,R3B581VNKYCP16,B00HFPOXM4,114966677,Garmin Vivofit Fitness Band,Wireless,5.0,5854.0,6063.0,N,Y,2014-03-12
1,US,49452274,RT0KPKVSQD0HI,B00A17IAO0,979081054,UP by Jawbone Wristband,Wireless,5.0,4856.0,5029.0,N,Y,2012-11-27
2,US,44086587,RYBUAAD9JZ1VW,B001S2RCWI,835787812,Garmin Portable Friction Dashboard Mount,Wireless,5.0,4017.0,4048.0,N,Y,2009-06-14
3,US,49452274,R2Z0F95XGL71C6,B00GOGV314,190508754,"UP24 by Jawbone Wristband, Retail Packaging",Wireless,5.0,3914.0,4022.0,N,N,2013-12-08
4,US,18464808,RZ0J3PVMPU4CJ,B00DGEGJ02,212863722,"Wemo Wi-Fi enabled, Works with Amazon Alexa",Wireless,1.0,3667.0,3894.0,N,Y,2013-11-01


In [3]:
data.dropna(subset=['marketplace','product_parent','product_title','product_category'],inplace=True,axis=0)
data = data.reset_index(drop=True)

In [4]:
data['marketplace'] = [re.sub(r'[^\w\s]', '', str(t)) for t in data['marketplace']]
data['product_parent'] = [re.sub(r'[^\w\s]', '', str(t)) for t in data['product_parent']]
data['product_title'] = [re.sub(r'[^\w\s]', '', str(t)) for t in data['product_title']]
data['product_category'] = [re.sub(r'[^\w\s]', '', str(t)) for t in data['product_category']]

In [5]:
data["combined"] = data['marketplace'] + '  ' + data['product_parent'] + ' ' + data['product_title'] + ' ' + data['product_category']
data.drop(['marketplace','product_parent','product_category'],axis=1,inplace=True)
data.head()

,customer_id,review_id,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_date,combined
0,51632887,R3B581VNKYCP16,B00HFPOXM4,Garmin Vivofit Fitness Band,5.0,5854.0,6063.0,N,Y,2014-03-12,US 114966677 Garmin Vivofit Fitness Band Wire...
1,49452274,RT0KPKVSQD0HI,B00A17IAO0,UP by Jawbone Wristband,5.0,4856.0,5029.0,N,Y,2012-11-27,US 979081054 UP by Jawbone Wristband Wireless
2,44086587,RYBUAAD9JZ1VW,B001S2RCWI,Garmin Portable Friction Dashboard Mount,5.0,4017.0,4048.0,N,Y,2009-06-14,US 835787812 Garmin Portable Friction Dashboa...
3,49452274,R2Z0F95XGL71C6,B00GOGV314,UP24 by Jawbone Wristband Retail Packaging,5.0,3914.0,4022.0,N,N,2013-12-08,US 190508754 UP24 by Jawbone Wristband Retail...
4,18464808,RZ0J3PVMPU4CJ,B00DGEGJ02,Wemo WiFi enabled Works with Amazon Alexa,1.0,3667.0,3894.0,N,Y,2013-11-01,US 212863722 Wemo WiFi enabled Works with Ama...


In [6]:
# Content Similarity
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(data["combined"])
cosine_similarities = linear_kernel(matrix,matrix)
p_title = data['product_title']
indices = pd.Series(data.index, index=data['product_title'])

In [7]:
def content_recommender(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1].any(), reverse=True) #truth value error
    sim_scores = sim_scores[1:31]
    product_indices = [i[0] for i in sim_scores]
    p_recommend = pd.merge(p_title.iloc[product_indices], data)
    return p_recommend.sort_values(by=['total_votes'], ascending=False)

In [8]:
content_recommender('Garmin Nuvi 1350T GPS with Traffic')

,product_title,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_date,combined
35,Garmin Vivofit Fitness Band,51632887,R3B581VNKYCP16,B00HFPOXM4,5.0,5854.0,6063.0,N,Y,2014-03-12,US 114966677 Garmin Vivofit Fitness Band Wire...
0,UP by Jawbone Wristband,49452274,RT0KPKVSQD0HI,B00A17IAO0,5.0,4856.0,5029.0,N,Y,2012-11-27,US 979081054 UP by Jawbone Wristband Wireless
4,Garmin Portable Friction Dashboard Mount,44086587,RYBUAAD9JZ1VW,B001S2RCWI,5.0,4017.0,4048.0,N,Y,2009-06-14,US 835787812 Garmin Portable Friction Dashboa...
5,UP24 by Jawbone Wristband Retail Packaging,49452274,R2Z0F95XGL71C6,B00GOGV314,5.0,3914.0,4022.0,N,N,2013-12-08,US 190508754 UP24 by Jawbone Wristband Retail...
8,Wemo WiFi enabled Works with Amazon Alexa,18464808,RZ0J3PVMPU4CJ,B00DGEGJ02,1.0,3667.0,3894.0,N,Y,2013-11-01,US 212863722 Wemo WiFi enabled Works with Ama...
10,LG Tone Wireless Bluetooth Stereo Headset Ret...,33924485,R3TIJPKMWLUP9Q,B0052YFYFK,5.0,3667.0,3726.0,N,Y,2011-09-11,US 97226392 LG Tone Wireless Bluetooth Stereo...
14,Garmin Nuvi 2595LMT,52215168,R1YY5H7O6P9NYQ,B005DIBHA6,4.0,3477.0,3593.0,N,N,2011-11-26,US 26681904 Garmin Nuvi 2595LMT Wireless
11,Garmin Nuvi 2595LMT,52215168,R1YY5H7O6P9NYQ,B005DIBHA6,4.0,3477.0,3593.0,N,N,2011-11-26,US 26681904 Garmin Nuvi 2595LMT Wireless
17,Misfit Shine Activity Monitor,7154102,R1CJRR7AVOIFK,B00FDWN8RM,5.0,3378.0,3468.0,N,Y,2014-06-22,US 526049974 Misfit Shine Activity Monitor Wi...
18,TomTom XXL 540TM 5Inch Widescreen Portable GPS...,41501220,R3J523QBQS9DV8,B003B3P2CY,4.0,3293.0,3323.0,N,Y,2010-05-14,US 778294373 TomTom XXL 540TM 5Inch Widescree...
